# Cluster and HII-regions Multi <a class="tocSkip">

the aim of this notebook is to combine the HII-region and cluster catalogues.
   
This notebook useses multiple galaxies at once.

In [ ]:
# reload modules after they have been modified
%load_ext autoreload
%autoreload 2

from pnlf.packages import *

from pnlf.constants import tab10, single_column, two_column
from pnlf.plot import quick_plot

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [ ]:
logging.basicConfig(stream=sys.stdout,
                    #format='(levelname)s %(name)s %(message)s',
                    datefmt='%H:%M:%S',
                    level=logging.INFO)

logger = logging.getLogger(__name__)


# first we need to specify the path to the raw data
basedir = Path('..')
data_ext = Path('a:') #basedir / 'data' / 'raw' 

sample_table = ascii.read(basedir/'..'/'pnlf'/'data'/'interim'/'sample.txt')
sample_table.add_index('name')
sample_table['SkyCoord'] = SkyCoord(sample_table['R.A.'],sample_table['Dec.'])
sample_table['power_index'] = 2.3
sample_table['power_index'][sample_table['AO'].mask]=2.8

## Read in data

In [ ]:
hst_sample      = set(['NGC0628','NGC1365','NGC1433', 'NGC1566', 'NGC3351', 'NGC3627', 'NGC4535'])
astrosat_sample = set([x.stem.split('_')[0] for x in (data_ext/'Astrosat').iterdir() if x.is_file() and x.suffix=='.fits'])
muse_sample     = set(sample_table['name'])
complete_sample = hst_sample & astrosat_sample & muse_sample

In [ ]:
# the original catalogue

with fits.open(basedir / 'data' / 'interim' / 'Nebulae_Catalogue_with_FUV_eq_v2p1.fits') as hdul:
    nebulae = Table(hdul[1].data)
nebulae['SkyCoord'] = SkyCoord(nebulae['cen_ra']*u.deg,nebulae['cen_dec']*u.deg,frame='icrs')

with np.errstate(divide='ignore'):
    nebulae['[SIII]/[SII]'] = np.nan
    SII = nebulae['SII6716_FLUX_CORR']+nebulae['SII6730_FLUX_CORR']
    SIII = nebulae['SIII6312_FLUX_CORR']+nebulae['SIII9068_FLUX_CORR']
    nebulae['[SIII]/[SII]'][SII>0] = SIII[SII>0]/SII[SII>0]
    nebulae['HA/FUV'] = nebulae['HA6562_FLUX_CORR']/nebulae['FUV_FLUX_CORR']

In [ ]:
# joined association/nebulae catalogue 
folder = basedir/'data'/'map_nebulae_association_v1'

lst = []
for file in folder.glob('*joined_v1.fits'):
    gal_name = file.stem.split('_')[0]
    print(f'reading {gal_name}')
    tbl = Table(fits.getdata(file,ext=1))
    tbl.add_column(gal_name,name='gal_name',index=0)
    lst.append(tbl)
catalogue = vstack(lst)

catalogue['SkyCoord_asc'] = SkyCoord(catalogue['Ra_asc'],catalogue['Dec_asc'])
catalogue['SkyCoord_neb'] = SkyCoord(catalogue['Ra_neb'],catalogue['Dec_neb'])
catalogue['HA/FUV'] = catalogue['HA6562_flux']/catalogue['FUV_flux']

print(f'{len(catalogue)} objects in final catalogue')

## Search for correlations

### FUV vs ionization

In [ ]:
from scipy.stats import binned_statistic, pearsonr, spearmanr

sample = astrosat_sample & muse_sample
filename = basedir/'reports'/'all_objects_FUV_over_SII'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 4
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())
    
#vmin,vmax = np.min(catalogue['HA6562_FLUX_CORR']),np.max(catalogue['HA6562_FLUX_CORR'])
vmin,vmax = 1e-16,1e-14
# loop over the galaxies we want to plot
for name in sorted(sample):  
    
    tmp = nebulae[(nebulae['gal_name']==name)]
        
    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    #catalogue = catalogue[catalogue['HA6562_FLUX']>np.nanpercentile(catalogue['HA6562_FLUX'],50)]
    tmp = tmp[tmp['FUV_FLUX_CORR']>3*tmp['FUV_FLUX_CORR_ERR']]
    tmp = tmp[tmp['SII6716_FLUX_CORR']>3*tmp['SII6716_FLUX_CORR_ERR']]
    tmp = tmp[tmp['SIII9068_FLUX_CORR']>3*tmp['SIII9068_FLUX_CORR_ERR']]

    r,p = spearmanr(tmp['[SIII]/[SII]'],tmp['HA/FUV'])
    print(f'{name}: rho={r:.2f}, {len(tmp)} objects')

    sc = ax.scatter(tmp['[SIII]/[SII]'],tmp['HA/FUV'],
               c=1e-20*tmp['HA6562_FLUX_CORR'],vmin=vmin,vmax=vmax,
               cmap=plt.cm.plasma,
               norm=mpl.colors.LogNorm(),
               s=1,marker='.')
    
    ax.text(0.05,0.9,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.75,0.15,r'$\rho$'+f'={r:.2f}',transform=ax.transAxes,fontsize=7)
    ax.text(0.62,0.05,f'{len(tmp):.0f} objects', transform=ax.transAxes,fontsize=7)
    
    ax.set(xscale='log',yscale='log',xlim=[1e-2,1],ylim=[2,4e2])
    # https://stackoverflow.com/questions/21920233/matplotlib-log-scale-tick-label-number-formatting/33213196
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    if i==nrows-1:
        ax.set_xlabel('[SIII]/[SII]')
    if j==0:
        ax.set_ylabel(r'H$\alpha$ / FUV')

        
for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    
    if i==0:
        #ax.remove()
        ax.axis('off')
        cbar = fig.colorbar(sc, ax=ax,
                            label=r'$\mathrm{H}\alpha$ / (erg s$^{-1}$ cm$^{-2}$ Hz$^{-1}$)',
                            orientation='horizontal')
    else:
        ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel('[SIII]/[SII]')


plt.savefig(filename.with_suffix('.png'),dpi=600)
plt.savefig(filename.with_suffix('.pdf'),dpi=600)

plt.show()


### Compare with HST ages

In [ ]:
from astropy.coordinates import match_coordinates_sky, search_around_sky
from scipy.stats import binned_statistic, pearsonr, spearmanr

# '[SIII]/[SII]' , 'HA/FUV', 'AGE_MINCHISQ', 'AGE_BAYES'
x_name, y_name = 'age', 'HA/FUV'
xlim = [0.5,10.5]
bins = 10
max_sep = 1*u.arcsec

#sample = set(np.unique(nebulae['gal_name'])) & hst_sample
sample = hst_sample & astrosat_sample & muse_sample

filename = basedir/'reports'/'all_objects_age_over_SII.pdf'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 3
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

# loop over the galaxies we want to plot
for name in sorted(sample): 

    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    tmp = catalogue[(catalogue['gal_name']==name) & (catalogue['mass']>1e3)]
    print(f'{name}: {len(tmp)} objects')
    
    mean, bin_edges, binnumber = binned_statistic(tmp[x_name],
                                                  tmp[y_name],
                                                  statistic='mean',
                                                  bins=bins,
                                                  range=xlim)
    std, _, _ = binned_statistic(tmp[x_name],
                                  tmp[y_name],
                                  statistic='std',
                                  bins=bins,
                                  range=xlim)

    ax.scatter(tmp[x_name],tmp[y_name],color='tab:blue',s=1)
    # plot the standard divation with yerr=std
    ax.errorbar((bin_edges[1:]+bin_edges[:-1])/2,mean,fmt='-')
    ax.text(0.65,0.85,f'{name}', transform=ax.transAxes,fontsize=7)

    ax.set(xlim=xlim)
    if i==nrows-1:
        ax.set_xlabel(f'{x_name.replace("_"," ")}')
    if j==0:
        ax.set_ylabel(f'{y_name.replace("_"," ")}')

for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(f'{x_name.replace("_"," ")}')


plt.savefig(filename,dpi=600)
plt.show()

In [ ]:
from astropy.coordinates import match_coordinates_sky, search_around_sky

# '[SIII]/[SII]' , 'HA/FUV'
x_name, y_name, z_name = '[SIII]/[SII]', 'HA/FUV', 'AGE_MINCHISQ'
xlim = [0.5,10.5]
bins = 10
max_sep = 2*u.arcsec

sample = muse_sample & hst_sample & astrosat_sample

filename = basedir/'reports'/'all_objects_FUV_over_SII_with_age.pdf'

#----------------------------------------------
# DO NOT MODIFY BELOW
#----------------------------------------------
ncols = 2
nrows = int(np.ceil(len(sample)/ncols))

if nrows*ncols<len(sample):
    raise ValueError('not enough subplots for selected objects') 
width = two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

# loop over the galaxies we want to plot
for name in sorted(sample): 

    # it makes a different if we match the clusters to the nebulae or the other way around
    catalogcoord = clusters[clusters['gal_name']==name].copy()
    matchcoord   = nebulae[nebulae['gal_name']==name].copy()

    idx, sep, _ = match_coordinates_sky(matchcoord['SkyCoord'],catalogcoord['SkyCoord'])

    catalogue = matchcoord.copy()

    for col in catalogcoord.columns:
        if col in catalogue.columns:
            catalogue[f'{col}2'] = catalogcoord[idx][col]
        else:
            catalogue[col] = catalogcoord[idx][col]

    catalogue['sep'] = sep
    catalogue = catalogue[sep.__lt__(max_sep)]
    catalogue = catalogue[~np.isnan(catalogue[x_name]) & ~np.isnan(catalogue[y_name]) & (catalogue['AGE_MINCHISQ']<10)]
    print(f'{name}: {len(catalogue)} objects in joined catalogue')

    # get the next axis and find position on the grid
    ax = next(axes_iter)
    if nrows>1 and ncols>1:
        i, j = np.where(axes == ax)
        i,j=i[0],j[0]
    elif ncols>1:
        i,j = 0, np.where(axes==ax)[0]
    elif nrows>1:
        i,j = np.where(axes==ax)[0],0
    else:
        i,j=0,0

    #catalogue = catalogue[catalogue['HA6562_FLUX']>np.nanpercentile(catalogue['HA6562_FLUX'],50)]
    #catalogue = catalogue[catalogue['FUV_FLUX_CORR']>3*catalogue['FUV_FLUX_CORR_ERR']]
    #catalogue = catalogue[catalogue['SII6716_FLUX_CORR']>3*catalogue['SII6716_FLUX_CORR_ERR']]
    #catalogue = catalogue[catalogue['SIII9068_FLUX_CORR']>3*catalogue['SIII9068_FLUX_CORR_ERR']]

    r,p = spearmanr(catalogue['[SIII]/[SII]'],catalogue['HA/FUV'])
    print(f'{name}: rho={r:.2f}, {len(catalogue)} objects')

    sc = ax.scatter(catalogue['[SIII]/[SII]'],catalogue['HA/FUV'],
                    c=catalogue[z_name],vmin=0, vmax=10,cmap=plt.cm.RdBu_r,
                    s=3,marker='.')
    
    ax.text(0.05,0.9,f'{name}', transform=ax.transAxes,fontsize=7)
    ax.text(0.7,0.15,r'$\rho$'+f'={r:.2f}',transform=ax.transAxes,fontsize=7)
    ax.text(0.55,0.05,f'{len(catalogue):.0f} objects', transform=ax.transAxes,fontsize=7)
    
    ax.set(xscale='log',yscale='log',xlim=[1e-2,1],ylim=[2,2e2])
    # https://stackoverflow.com/questions/21920233/matplotlib-log-scale-tick-label-number-formatting/33213196
    ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))
    ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y, _: '{:g}'.format(y)))

    if i==nrows-1:
        ax.set_xlabel('[SIII]/[SII]')
    if j==0:
        ax.set_ylabel(r'H$\alpha$ / FUV')

fig.colorbar(sc,ax=axes.ravel().tolist(),label=f'{z_name.replace("_"," ")}')
        
for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

    # add the xlabel to the axes above
    axes[nrows-2,ncols-1-i].set_xlabel(f'{x_name.replace("_"," ")}')


plt.savefig(filename,dpi=600)
plt.show()


### with association ages

In [ ]:
tmp = catalogue[catalogue['mass']>1e3]

In [ ]:
from cluster.auxiliary import bin_stat

xlim = [0.5,10.5]

fig,ax=plt.subplots(figsize=(6,4))
ax.errorbar(tmp['age'],tmp['HA/FUV'],fmt='o')
            #xerr=tmp['age_err'],yerr=tmp['HA/FUV_err'])

x,mean,std = bin_stat(tmp['age'],tmp['HA/FUV'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='Age/Myr',ylabel='Halpha / FUV',xlim=xlim)
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

plt.savefig(basedir/'reports'/f'all_galaxies_HaFUV_over_age.pdf',dpi=600)
plt.show()

In [ ]:
xlim = [0.5,10.5]

fig,ax=plt.subplots(figsize=(6,4))
sc= ax.scatter(tmp['age'],tmp['eq_width']) #,c=tmp['mass'],vmin=1e5,vmax=3e5)
#fig.colorbar(sc,label='mass / Msun')

x,mean,std = bin_stat(tmp['age'],tmp['eq_width'],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black',label='mass / Msun')
ax.set(xlabel='Age/Myr',ylabel='equivalent width / Angstrom',xlim=xlim)
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')

plt.savefig(basedir/'reports'/f'all_galaxies_eq_width_over_age.pdf',dpi=600)
plt.show()

In [ ]:
xlim = [0,100]
criteria = catalogue['HA/FUV']<150

fig,ax=plt.subplots(figsize=(6,4))
sc= ax.scatter(catalogue['eq_width'][criteria],catalogue['HA/FUV'][criteria])

x,mean,std = bin_stat(catalogue['eq_width'][criteria],catalogue['HA/FUV'][criteria],xlim)
ax.errorbar(x,mean,yerr=std,fmt='-',color='black')
ax.set(xlabel='equivalent width / Angstrom',ylabel='Halpha / FUV',xlim=xlim,ylim=(0,70))
#ax.set_title(r'only clusters with $M>10^{5}M_\odot$')
#plt.savefig(basedir/'reports'/name/f'{name}_HaFUV_over_eq_width.pdf',dpi=600)
plt.show()

In [ ]:
from mpl_toolkits.axes_grid1 import make_axes_locatable

# we expect EBV_balmer = 2 EBV_stars

fig = plt.figure(figsize=(single_column,single_column/1.1))
ax = fig.add_subplot()
divider = make_axes_locatable(ax)
cax = divider.append_axes('right', size="10%", pad=0.2,)

xlim = [0,10]
sc=ax.scatter(tmp['EBV_stars'],tmp['EBV_balmer'],c=tmp['age'],s=3,vmin=0,vmax=10,cmap=plt.cm.viridis_r)
ax.plot([0,1],[0,2],color='black')
ax.plot([0,2],[0,2],color='black')
fig.colorbar(sc,label='age / Myr',cax=cax)
ax.set(xlim=[0,0.7],ylim=[0,0.7],xlabel='E(B-V) stars',ylabel='E(B-V) Balmer')

plt.savefig(basedir/'reports'/f'all_galaxies_EBV_Balmer_vs_Stars.pdf',dpi=600)
plt.show()

In [ ]:
from cluster.plot import corner


filename = basedir/'reports'/f'all_galaxies_corner'
columns  = ['age','HA/FUV','eq_width','met_scal','logq_D91']
limits   = {'age':(0,10),'eq_width':(0,100),'HA/FUV':(0,50),'HA/NUV':(0,20),'met_scal':(8.4,8.7),'logq_D91':(6,8)}

tmp = catalogue[catalogue['contained']]
print(f'sample contains {len(tmp)} objects')

corner(tmp,columns,limits,nbins=5,filename=filename,vmin=1000,vmax=1e6)

## Compare all with Starburst99

In [ ]:
from starburst import Cluster

cluster = Cluster(stellar_model='GENEVAv40',metallicity=0.014)

In [ ]:
from tqdm import tqdm

catalogue['Qpredicted'] = np.nan
for row in tqdm(catalogue):
    # the masses in the catalogue are off by a factor of 10
    mass = row['mass']
    age  = row['age']*u.Myr
    scaled_cluster = cluster.scale(mass)
    idx = np.argmin(np.abs(scaled_cluster.quanta['Time']-age))
    row['Qpredicted'] = scaled_cluster.quanta['HI_rate'][idx].value
    
catalogue['distance'] = np.nan
for gal_name in catalogue['gal_name']:
    distance = Distance(distmod=sample_table.loc[gal_name]['(m-M)'])
    catalogue['distance'][catalogue['gal_name']==gal_name] = distance

the conversion factor is from Niederhofer+2016 (or Kennicutt+98)

$$
Q(\mathrm{H}^0) = 7.31\cdot 10^{11} L(\mathrm{H}\alpha)
$$

we only use a subsample (high mass, contained, speparated etc.)

In [ ]:
catalogue['L(Ha)'] = (catalogue['HA6562_flux']*1e-20*u.erg/u.s/u.cm**2 *4*np.pi*(catalogue['distance']*u.Mpc)**2).to(u.erg/u.s)
catalogue['Qobserved'] = 7.31e11*catalogue['L(Ha)']/u.erg

criteria = (catalogue['mass']>1e3) & (catalogue['age']<6) & (catalogue['age']>0) & (catalogue['contained'])
tmp = catalogue[criteria]
fesc = (tmp['Qpredicted']-tmp['Qobserved'])/tmp['Qpredicted']

print(f'fesc={np.nanmean(fesc):.2f} (from {np.sum(criteria)} matches)')
print(f"{np.sum(fesc<0)} of {len(tmp)} regions have negative fesc")

In [ ]:
fig,ax =plt.subplots(figsize=(single_column,single_column))

print(f'fesc={np.nanmean(fesc):.2f}')
ax.hist(fesc,bins=np.arange(0,1.1,0.05))
ax.set(xlim=[0,1.1],xlabel='fesc')
plt.show()

In [ ]:
fig,ax =plt.subplots(figsize=(single_column,0.8*single_column))

print(f'fesc={np.mean(fesc):.2f}')

Qpredicted = np.logspace(48,52)
cmap = plt.cm.get_cmap('copper',6)
lines = ["-","--","-.",":"]
for i,f in enumerate([0.0,0.5,0.9,0.99]):
    Qobserved = Qpredicted*(1-f)
    ax.plot(np.log10(Qpredicted),np.log10(Qobserved),ls=lines[i],c='k',label=f'$f_\mathrm{{esc}}={f}$',zorder=1)

sc=ax.scatter(np.log10(tmp['Qpredicted']),np.log10(tmp['Qobserved']),
              c=tmp['logq_D91'],cmap=cmap,vmin=5,vmax=8,s=2,zorder=2)
ax.legend()
fig.colorbar(sc,label='$\log_{10} q$ (from Diaz+91)')
#ax.plot([0,100],np.array([10,110]),color='gray',ls='--')
#ax.plot([0,100],np.array([-10,90]),color='gray',ls='--')

ax.set(xlabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ predicted (SB99)',
       ylabel=r'$\log_{10} (\mathcal{Q (\mathrm{H}^0)} / \mathrm{s}^{-1})$ observed (MUSE)',
       xlim=[48,52],ylim=[48,52])
#ax.xaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y,pos: ('{{:.{:1d}f}}'.format(int(np.maximum(-np.log10(y),0)))).format(y)))
#ax.yaxis.set_major_formatter(mpl.ticker.FuncFormatter(lambda y,pos: ('{{:.{:1d}f}}'.format(int(np.maximum(-np.log10(y),0)))).format(y)))
ax.xaxis.set_major_locator(mpl.ticker.MaxNLocator(4)) 
ax.yaxis.set_major_locator(mpl.ticker.MaxNLocator(4)) 

plt.savefig(basedir/'reports'/f'escape_fraction_v2.pdf',dpi=600)
plt.show()

In [ ]:
# calculate Ha on a grid of different ages/masses with Starburst99
age_grid = np.array([1,3,5,7,9])*u.Myr
mass_grid = np.logspace(2,6,10)
ionizing_photons = np.zeros((len(age_grid),len(mass_grid)))
for i,mass in enumerate(mass_grid):
    scaled_cluster = cluster.scale(mass)
    for j,age in enumerate(age_grid):
        idx = np.argmin(np.abs(scaled_cluster.quanta['Time']-age))
        ionizing_photons[j,i] = scaled_cluster.quanta['HI_rate'][idx].value
distance = 10*u.Mpc
LHa = ionizing_photons / 7.31e11 * u.erg 
FHa = LHa / (1e-20*u.erg/u.s/u.cm**2 *4*np.pi*distance**2).to(u.erg/u.s)

In [ ]:
nrows, ncols = 2,4

cmap = plt.cm.get_cmap('copper', 5)
fig=plt.figure(figsize=(two_column,nrows/ncols*two_column))
axes = []
for i,gal_name in enumerate(np.unique(tmp['gal_name'])):
    ax = fig.add_subplot(nrows,ncols,i+1)
    axes.append(ax)
    sub = tmp[tmp['gal_name']==gal_name]
    
    distance = sub['distance'][0]*u.Mpc
    LHa = ionizing_photons / 7.31e11 * u.erg 
    FHa = LHa / (u.erg/u.s/u.cm**2 *4*np.pi*distance**2).to(u.erg/u.s)
    
    sc=ax.scatter(sub['mass'],1e-20*sub['HA6562_flux'],s=3,
               c=sub['age'],cmap=cmap,vmin=0,vmax=10,zorder=2)
    
    for j,age in enumerate(age_grid):
        color = cmap(age.value/10)
        ax.plot(mass_grid,FHa[j,:],label=age,color=color)
    ax.set(xlim=[1e3,1e5],ylim=[1e-17,1e-12],xscale='log',yscale='log')
    ax.text(0.05,0.9,f'{gal_name}', transform=ax.transAxes,fontsize=7)

    if i%ncols==0:
        ax.set(ylabel=r'$F(\mathrm{H}\alpha)$ / erg s$^{-1}$ cm$^{-2}$')
    else:
        ax.set_yticklabels([])
    if i//ncols==nrows-1:
        ax.set(xlabel=r'mass / M$_\odot$')
    else:
        ax.set_xticklabels([])

# Create the legend
h,l = axes[0].get_legend_handles_labels()
fig.legend(h,l,
           ncol=5,
           loc="upper center",   # Position of legend
           borderaxespad=0.1,    # Small spacing around legend box
           )
plt.tight_layout()
fig.subplots_adjust(right=0.9)
cbar_ax = fig.add_axes([0.93, 0.11, 0.02, 0.84])
fig.colorbar(sc,cax=cbar_ax,label='age / Myr')

plt.savefig(basedir/'reports'/'mass_vs_Halpha_v2.pdf',dpi=600)
plt.show()

### separation and age

In [ ]:
sep = catalogue['SkyCoord_asc'].separation(catalogue['SkyCoord_neb']).to(u.arcsec).value

fig,ax=plt.subplots()
bins = np.arange(0,10)

print(f"x<0.5: {np.mean(catalogue[sep<0.5]['age']):.2f}")
print(f"0.5<x<1: {np.mean(catalogue[(sep>0.5) & (sep<1)]['age']):.2f}")
print(f"1<x: {np.mean(catalogue[sep>1]['age']):.2f}")

ax.hist(catalogue[sep<0.5]['age'],bins=bins,alpha=0.5,label=r'$x<0.5"$')
ax.hist(catalogue[(sep>0.5) & (sep<1)]['age'],bins=bins,alpha=0.5,label=r'$0.5"<x<1"$')
ax.hist(catalogue[sep>1]['age'],bins=bins,alpha=0.5,label=r'$1"<x$')

ax.legend()
ax.set(xlim=[0,10])
plt.show()

In [ ]:
fig,ax=plt.subplots()

ax.scatter(catalogue['HA6562_flux'],catalogue['age'])

plt.show()

## Plot regions (MUSE & HST over WFI)

In [ ]:
from astropy.nddata import Cutout2D
from cluster.regions import find_sky_region

sample = set([x.stem.split('_')[0].upper() for x in (data_ext/'HST'/'white_light').iterdir()])
sample = muse_sample
ncols = 5
nrows = int(np.ceil(len(sample)/ncols))

width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for name in sorted(sample):
    
    print(name)
    
    if name in hst_sample:
        with fits.open(data_ext / 'HST' / 'white_light' / f'{name.lower()}_white_24rgb.fits') as hdul:
            hst_whitelight = NDData(hdul[0].data,mask=hdul[0].data==0,meta=hdul[0].header,wcs=WCS(hdul[0].header))
            hst_whitelight.data[hst_whitelight.data==0] = np.nan
        
    filename = data_ext / 'MUSE_DR2' / 'MUSEDAP' / f'{name}_MAPS.fits'
    with fits.open(filename) as hdul:
        Halpha = NDData(data=hdul['HA6562_FLUX'].data,
                        uncertainty=StdDevUncertainty(hdul['HA6562_FLUX_ERR'].data),
                        mask=np.isnan(hdul['HA6562_FLUX'].data),
                        meta=hdul['HA6562_FLUX'].header,
                        wcs=WCS(hdul['HA6562_FLUX'].header))
        
    filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
    with fits.open(filename) as hdul:

        WFI = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=WCS(hdul[0].header))
        

    reg_muse_pix, reg_muse_sky = find_sky_region(Halpha.mask.astype(int),wcs=Halpha.wcs)
    if name in hst_sample:
        reg_hst_pix, reg_hst_sky = find_sky_region(hst_whitelight.mask.astype(int),wcs=hst_whitelight.wcs)
    
    WFI_cutout = Cutout2D(WFI.data,sample_table.loc[name]['SkyCoord'],size=8*u.arcmin,wcs=WFI.wcs)
    
    # project from muse to hst coordinates
    reg_muse_wfi = reg_muse_sky.to_pixel(WFI_cutout.wcs)
    if name in hst_sample:
        reg_hst_wfi  = reg_hst_sky.to_pixel(WFI_cutout.wcs)

    ax = next(axes_iter)

    # plot image
    norm = simple_norm(WFI_cutout.data,clip=False,percent=99)
    ax.imshow(WFI_cutout.data,norm=norm,cmap=plt.cm.gray,origin='lower')

    reg_muse_wfi.plot(ax=ax,ec='tab:red',label='MUSE',lw=0.5)
    if name in hst_sample:
        reg_hst_wfi.plot(ax=ax,ec='tab:orange',label='HST',lw=0.5)
    t = ax.text(0.05,0.91,name, transform=ax.transAxes,color='black',fontsize=8)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    ax.set_xticks([])
    ax.set_yticks([])

for i in range(nrows*ncols-len(sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=-0.01,hspace=0.05)
plt.savefig(basedir/'reports'/'all_objects.pdf',dpi=600)
plt.savefig(basedir/'reports'/'all_objects.png',dpi=600)

plt.show()

In [ ]:
from astropy.nddata import Cutout2D
from cluster.regions import find_sky_region


ncols = 5
nrows = int(np.ceil(len(muse_sample)/ncols))

width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for name in sorted(muse_sample):
    
    print(name)
    
    catalogue_file = basedir/'..'/'PNLF'/'data'/'catalogues'/f'{name}_nebulae.txt'
    catalogue = ascii.read(catalogue_file,format='fixed_width_two_line',delimiter_pad=' ',position_char='=')
    catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
    catalogue=catalogue[catalogue['type']=='PN']
    
    filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
    with fits.open(filename) as hdul:

        WFI = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=WCS(hdul[0].header))
        
    
    WFI_cutout = Cutout2D(WFI.data,sample_table.loc[name]['SkyCoord'],size=5*u.arcmin,wcs=WFI.wcs)

    ax = next(axes_iter)

    # plot image
    norm = simple_norm(WFI_cutout.data,clip=False,percent=99)
    ax.imshow(WFI_cutout.data,norm=norm,cmap=plt.cm.gray,origin='lower')
    
    x,y = catalogue['SkyCoord'].to_pixel(WFI_cutout.wcs)
    ax.scatter(x,y,marker='o',ec='tab:red',fc='none',s=1,lw=0.2)
    
    t = ax.text(0.05,0.91,name, transform=ax.transAxes,color='black',fontsize=8)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    ax.set_xticks([])
    ax.set_yticks([])

for i in range(nrows*ncols-len(muse_sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=-0.01,hspace=0.05)
plt.savefig(basedir/'reports'/'all_objects_PN.pdf',dpi=600)
plt.show()

In [ ]:
from astropy.nddata import Cutout2D
from cluster.regions import find_sky_region


ncols = 5
nrows = int(np.ceil(len(muse_sample)/ncols))

width = 1.5*two_column
fig, axes = plt.subplots(nrows=nrows,ncols=ncols,figsize=(width,width/ncols*nrows))
axes_iter = iter(axes.flatten())

for name in sorted(muse_sample):
    
    print(name)
    
    catalogue = filter_table(nebulae,gal_name=name)
    #catalogue['SkyCoord'] = SkyCoord(catalogue['RaDec'])
    
    filename = data_ext / 'WFI' / f'{name}_Rc_flux_nosky.fits'
    with fits.open(filename) as hdul:
        WFI = NDData(data=hdul[0].data,
                     meta=hdul[0].header,
                     wcs=WCS(hdul[0].header))
        
    
    WFI_cutout = Cutout2D(WFI.data,sample_table.loc[name]['SkyCoord'],size=5*u.arcmin,wcs=WFI.wcs)

    ax = next(axes_iter)

    # plot image
    norm = simple_norm(WFI_cutout.data,clip=False,percent=99)
    ax.imshow(WFI_cutout.data,norm=norm,cmap=plt.cm.gray,origin='lower')
    
    x,y = catalogue['SkyCoord'].to_pixel(WFI_cutout.wcs)
    ax.scatter(x,y,marker='o',ec='tab:blue',fc='none',s=1,lw=0.2)
    
    t = ax.text(0.05,0.91,name, transform=ax.transAxes,color='black',fontsize=8)
    t.set_bbox(dict(facecolor='white', alpha=1, ec='white'))

    ax.set_xticks([])
    ax.set_yticks([])

for i in range(nrows*ncols-len(muse_sample)):

    # remove the empty axes at the bottom
    ax = next(axes_iter)
    ax.remove()

plt.subplots_adjust(wspace=-0.01,hspace=0.05)
plt.savefig(basedir/'reports'/'all_objects_nebulae.pdf',dpi=600)
plt.show()

## Playground